<a href="https://colab.research.google.com/github/Svein-Tore/colab/blob/main/Lambda_tuning_interaktiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interaktiv λ‑tuning for FOPDT‑prosess (PI‑regulator)

Denne notatboken lar deg utforske **Lambda-/Skogestad‑metoden** ved å justere prosessparametrene **K**, **T**, **L** og ønsket lukket sløyfe‑tidskonstant **λ** med sliders.

**Formler (fra undervisningsopplegget):**
- $K_p = \dfrac{T}{K(\lambda + L)}$
- $T_i = \min\big(T,\;4(\lambda + L)\big)$

Kjør cellene fra toppen og bruk sliderne til å se hvordan **PV** og **pådrag u** endrer seg.

In [ ]:

# === Colab-oppsett (kun ved behov) ===
import sys
try:
    import ipywidgets  # noqa: F401
except ImportError:
    !pip install -q ipywidgets
    import ipywidgets  # noqa: F401

# Aktiver Colabs widget-manager dersom vi er i Colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    print("Kjører ikke i Colab (hopper over enable_custom_widget_manager).", file=sys.stderr)

# === Interaktiv λ-tuning for FOPDT (PI) – med setpunkt i prosent ===
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import FloatSlider, HBox, VBox, Output

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11

def simulate_fopdt_pi(K=2.904, T=125.4, L=5.2, lam=62.7,
                      SP_unit=1.0, dt=0.1, T_end=300.0, u_clip=2.0):
    """FOPDT + dødtid + PI (lambda-tunet). Returnerer time, sp (unit), y (unit), u, Kp, Ti."""
    # Lambda-/Skogestad-formler
    Kp = T / (K * (lam + L))
    Ti = min(T, 4.0 * (lam + L))

    time = np.arange(0.0, T_end + dt, dt)
    sp   = np.full_like(time, SP_unit, dtype=float)  # konstant settpunkt (i enhetsverdi 0..1)

    y = np.zeros_like(time)
    u = np.zeros_like(time)
    e_int = 0.0

    delay_steps = int(np.round(L/dt))
    u_buf = np.zeros(delay_steps + 1)

    for k in range(1, len(time)):
        e = sp[k] - y[k-1]
        e_int += e * dt
        u[k] = Kp * (e + (e_int / Ti))
        if u_clip is not None:
            u[k] = np.clip(u[k], -abs(u_clip), abs(u_clip))

        if delay_steps > 0:
            u_buf = np.roll(u_buf, 1)
            u_buf[0] = u[k]
            u_delay = u_buf[-1]
        else:
            u_delay = u[k]

        # FOPDT: dy/dt = (-y + K*u_delay)/T  (Euler-diskretisering)
        dy = (-y[k-1] + K * u_delay) / T
        y[k] = y[k-1] + dy * dt

    return time, sp, y, u, Kp, Ti

# Standardverdier (fra nivå-eksempelet)
K0, T0, L0 = 2.904, 125.4, 5.2
lam0 = T0/2
SP0_percent = 100.0  # standard SP = 100%

# Sliders
K_sl    = FloatSlider(value=K0,  min=0.1,  max=10.0,  step=0.01, description='K')
T_sl    = FloatSlider(value=T0,  min=5.0,  max=500.0, step=0.1,  description='T [s]')
L_sl    = FloatSlider(value=L0,  min=0.0,  max=60.0,  step=0.1,  description='L [s]')
lam_sl  = FloatSlider(value=lam0,min=0.1,  max=400.0, step=0.1,  description='λ [s]')

# Settpunkt i prosent (0..100)
sp_sl   = FloatSlider(value=SP0_percent, min=0.0, max=100.0, step=1.0, description='SP [%]')

dt_sl    = FloatSlider(value=0.1,  min=0.02, max=0.5,   step=0.01, description='Δt [s]')
Tend_sl  = FloatSlider(value=300., min=30.0, max=1200., step=10.0, description='T_end [s]')
uclip_sl = FloatSlider(value=2.0,  min=0.5,  max=5.0,   step=0.1,  description='|u|max')

out_fig = Output()

def update_plot(change=None):
    with out_fig:
        out_fig.clear_output(wait=True)

        # Konverter SP fra % til enhetsverdi 0..1
        SP_unit = sp_sl.value / 100.0

        time, sp_unit, y_unit, u, Kp, Ti = simulate_fopdt_pi(
            K_sl.value, T_sl.value, L_sl.value,
            lam_sl.value, SP_unit, dt_sl.value, Tend_sl.value,
            u_clip=uclip_sl.value
        )

        # For visning i %:
        sp_pct = sp_unit * 100.0
        y_pct  = y_unit  * 100.0

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7.2, 6), sharex=True,
                                       gridspec_kw={'height_ratios': [2.0, 1.0]})

        # PV-plot i %
        ax1.plot(time, sp_pct, 'k--', lw=1.2, label='SP [%]')
        ax1.plot(time, y_pct,  lw=2.0, label='PV [%]')
        ax1.set_ylabel('PV / SP [%]')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='lower right')
        ax1.set_title('FOPDT + PI (λ‑tuning)  |  '
                      f'Kp={Kp:.3f}, Ti={Ti:.1f}s  |  '
                      f'λ={lam_sl.value:.1f}s, T={T_sl.value:.1f}s, L={L_sl.value:.1f}s, '
                      f'SP={sp_sl.value:.0f}%')

        # Pådrag (fortsatt i samme enheter som før)
        ax2.plot(time, u, lw=1.6)
        ax2.set_ylabel('u [–]')
        ax2.set_xlabel('Tid [s]')
        ax2.grid(True, alpha=0.3)
        plt.show()

# Koble alle sliders til plott
for w in (K_sl, T_sl, L_sl, lam_sl, sp_sl, dt_sl, Tend_sl, uclip_sl):
    w.observe(update_plot, names='value')

# Tegn første gang og vis layout
update_plot()
display(
    VBox([
        HBox([K_sl, T_sl]),
        HBox([L_sl, lam_sl]),
        HBox([sp_sl]),                 # SP-slider i egen rad (i %)
        HBox([dt_sl, Tend_sl, uclip_sl]),
        out_fig
    ])
)
